- 展示如何使用 OpenAI 的 GPT 模型與 Llama 模型處理技術問題，特別是結合流式傳輸與靜態回應的展示。

In [15]:
from openai import OpenAI
import ollama
from dotenv import load_dotenv
import os
# 用於在 Jupyter Notebook 中實時顯示與更新 Markdown 格式的內容。
from IPython.display import Markdown, display, update_display


# NOTE: 選擇模型與加載 API 金鑰
MODEL_GPT = 'gpt-4o-mini'  # 選擇 GPT 模型
MODEL_LLAMA = 'llama3.2'   # 選擇 Llama 模型

# 載入環境變數
load_dotenv()
openai = OpenAI()


# NOTE: 定義問題與系統提示
# 問題內容
question = """
請解釋這段程式碼的作用和原因：
yield from {book.get("author") for book in books if book.get("author")}
"""
# 系統提示
system_prompt = "您是一位樂於助人的技術導師，回答有關 python 代碼、軟體工程、數據科學和法學碩士的問題"
# 用戶輸入提示
user_prompt = "請詳細解釋以下問題：" + question

# 組裝對話訊息
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [16]:
# TODO: GPT 模型的流式請求
# 創建流式請求
stream = openai.chat.completions.create(model=MODEL_GPT, messages=messages, stream=True)

# 初始化回應變數
response = ""
display_handle = display(Markdown(""), display_id=True)

# 處理流式數據
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```", "").replace("markdown", "")  # 清理多餘格式
    update_display(Markdown(response), display_id=display_handle.display_id)

這段程式碼在 Python 中使用了 `yield from`，這是一個生成器的語法，用來從一個可迭代的對象中逐個返回元素。在這個特定的代碼片段中，`yield from` 後面使用了一個集合推導式（set comprehension）來生成一個集合。

我們來逐部分解析這段程式碼的作用：

1. **集合推導式**：`{book.get("author") for book in books if book.get("author")}`  
   - `books` 是一個可迭代對象（例如列表），包含許多書籍的資料，通常這些資料以字典的形式存儲。
   - `book.get("author")` 會從每個 `book` 字典中提取出 `"author"` 鍵對應的值。
   - `if book.get("author")` 是一個條件語句，確保只有在 `book` 字典中存在 `"author"` 這一鍵，且其值不為 `None` 或空字符串的情況下，才會將這個作者名添加到集合中。
   - 最終，這部分代碼會生成一個集合，這個集合只包含 `books` 中所有有效的作者名（即排除掉沒有作者的書籍）。

2. **`yield from`**：  
   - `yield from` 的作用是簡化從子生成器（也就是可迭代對象，如列表或集合）中返回值的過程。
   - 在這裡，`yield from` 將會對上面生成的集合進行迭代，並逐個返回集合中的每一個作者名。

### 總結
因此，整段程式碼的作用是：從一組書籍中提取所有有效的作者名，並將它們逐個返回作為生成器的輸出。這樣的做法有以下幾個好處：

- **去重**：因為集合會自動去除重複的作者名。
- **懶加載**：使用生成器可以提高性能，特別是在處理大量數據時，因為它會在需要時才生成下一个值，而不是一次性生成所有值。

示例用法可能如下：
python
def authors_generator(books):
    yield from {book.get("author") for book in books if book.get("author")}

books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author B"},
    {"title": "Book 3", "author": "Author A"},  # 重複的作者
    {"title": "Book 4"}  # 沒有作者
    # 其他書籍...
]

for author in authors_generator(books):
    print(author)

這段代碼將會輸出 `Author A` 和 `Author B`，並且不會重複列印同一個作者名。

In [17]:
# TODO: Llama 模型的回應處理
# 使用 Llama 模型進行回應
response = ollama.chat(model=MODEL_LLAMA, messages=messages)

# 提取回應內容
reply = response['message']['content']

# 顯示回應
display(Markdown(reply))

我們來一步步分析這段程式碼：

```python
yield from {book.get("author") for book in books if book.get("author")}
```

這是一個 Python 3.3+ 的語法，使用了 `yield from` 的機制。

**什么是 yield from ?**

`yield from` 是一個可以讓我們將另一個 iterator 選項的 iterator 將其內容轉換成 generator 的 keyword。它的主要目的是使我們能夠透過其他 iterator 進行 iteration，從而減少 code 重複。

**分解這段程式碼**

現在，讓我們一步步分解這段程式碼：

1. `{book.get("author") for book in books if book.get("author")}` 是一個 generator expression。
	* `for book in books` 進行 iteration 在 `books` 中找尋符合條件的 book。
	* `if book.get("author")` 會對每個 book 进行 filters，從而保留包含 "author" 欄位的書籍。
2. `yield from` 這個 keyword 則將該 generator expression 的 iterator 將其內容轉換成另一個 generator。
3. 這樣做的結果是，我們得到了一個新的 iterator，它會透過所有符合條件的 book 找尋其 "author" 欄位的值。

**目的和原因**

這段程式碼的目的是將 book 的 "author" 欄位值轉換成一個 generator iterable。這通常會被用於下面幾種情況：

* 這些值可能需要透過某個 iterator 进行處理或過濾。
* 這些值可能需要透過另一個生成器進行 iteration。
* 這樣做的結果是，程式碼更簡潔、更可讀。

例如，如果我們有以下 `books` list：

```python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": None},
    {"title": "Book 3", "author": "Author C"}
]
```

透過這段程式碼，我們可以得到一個 iterator，從而對每個 book 找尋其 "author" 欄位的值。這樣做的結果是：

```python
result = {book.get("author") for book in books if book.get("author")}
print(result)  # Output: {"Author A", "Author C"}
```

總之，這段程式碼通過使用 `yield from` 來將 generator expression 的 iterator 將其內容轉換成另一個 generator，從而簡化程式碼並提高效率。

## 設定API_KET跟使用
- 要先把API_KET放到.env
```
OPENAI_API_KEY=xxxx
ANTHROPIC_API_KEY=xxxx
GOOGLE_API_KEY=xxxx
```

In [11]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import ollama
# import anthropic # cloud
# import google.generativeai # gemini
from IPython.display import Markdown, display, update_display

In [4]:
# TODO: 載入環境便數
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
# anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
# google_api_key = os.getenv('GOOGLE_API_KEY')

# TODO: 連線到模型
openai = OpenAI()
# claude = anthropic.Anthropic()
# google.generativeai.configure()

## 嘗試讓大型語言模型講笑話

In [5]:
system_message = "你是一個擅長講笑話的助手"
user_prompt = "為數據科學家觀眾講一個輕鬆的笑話"

prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
]


- 調用模型

In [8]:
# GPT-3.5-Turbo
completion = openai.chat.completions.create(model='gpt-3.5-turbo', messages=prompts)
print(completion.choices[0].message.content)

為數據科學家觀眾講一個笑話：

為什麼數據科學家不喜歡去海灘放鬆呢？因為他們總是在思考如何把海浪波形轉換成數據模型！


In [ ]:
# GPT-4o-mini
completion = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompts,
    # temperature=0.7 # 控制生成內容的隨機性(0-1)
)
print(completion.choices[0].message.content)

為數據科學家講的笑話：

為什麼數據科學家總是帶著一把鋸子？

因為他們想要"切割"數據！


In [9]:
# GPT-4o
completion = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    # temperature=0.4
)
print(completion.choices[0].message.content)

為什麼數據科學家總是帶著眼鏡？

因為他們需要用 "R" 看清東西！


In [ ]:
# Claude 3.5 Sonnet
# API needs system message provided separately from user prompt

# message = claude.messages.create(
#     model="claude-3-5-sonnet-20240620",
#     max_tokens=200,
#     temperature=0.7,
#     system=system_message,
#     messages=[
#         {"role": "user", "content": user_prompt},
#     ],
# )

# print(message.content[0].text)

In [ ]:
# gemini

# gemini = google.generativeai.GenerativeModel(
#     model_name='gemini-1.5-flash',
#     system_instruction=system_message
# )
# response = gemini.generate_content(user_prompt)
# print(response.text)

In [ ]:
# 使用stream形式調用gpt-4o
tream = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.7,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

## Chatbot 多輪對話結構設計

- 對話歷史是以 列表形式 表示，每個元素是一個字典 (JSON 格式)。
- 每個字典包含兩個關鍵屬性：
    - role：表示發言角色，例如 system(設定模型的上下文)、user 或 assistant(模型自身)。
    - content：角色發送的具體訊息內容。
- 範例
```json
[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What's the weather today?"},
    {"role": "assistant", "content": "It's sunny and warm."},
    {"role": "user", "content": "What about tomorrow?"}
]
```

In [15]:
import ollama
# 定義 GPT 和 Ollama 模型
gpt_model = "gpt-4o-mini"
ollama_model = "llama3.2"

# 定義 GPT 和 Ollama 的角色
gpt_system = "你是一個非常好爭論的聊天機器人；你不同意談話中的任何事情，並對一切提出質疑。"
ollama_system = "你是一個非常有禮貌、有禮貌的聊天機器人。你試著同意對方所說的一切，或找到共同點。如果對方不高興，你試著讓他們平靜下來並繼續聊天。"

# 定義初始訊息
gpt_messages = ["Hi there"]
ollama_messages = ["Hi"]

# 定義 GPT 回應函數
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, ollama in zip(gpt_messages, ollama_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": ollama})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

# 定義 Ollama 回應函數
def call_ollama():
    messages = [{"role": "system", "content": ollama_system}]
    for gpt, ollama in zip(gpt_messages, ollama_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": ollama})
    response = ollama.chat(model=ollama_model, messages=messages)
    return response['message']['content']

# 模擬對話
gpt_reply = call_gpt()
ollama_reply = call_ollama()

print("GPT Reply:", gpt_reply)
print("Ollama Reply:", ollama_reply)


AttributeError: 'str' object has no attribute 'chat'